# data

In [1]:
from data import get_original_data, PosFeature, NegFeature
import numpy as np

import numpy as np
import pandas as pd
import sys
import seaborn as sns
from numpy import random

from matplotlib import pyplot as plt
import sys
import time
import math
from sklearn.preprocessing import StandardScaler
import optuna
import torch
from torch import nn, optim
from torch.nn import functional as F
from torch.optim.lr_scheduler import MultiStepLR
from torch.utils.data import Dataset, DataLoader


/Users/hejizhang/anaconda3/envs/py39/lib/python3.9/site-packages/ovito/_extensions/anari.py:2: UserWarning: Did you accidentally install the OVITO package from the PyPI repository in an Anaconda/Miniconda Python interpreter using the 'pip' command? This will likely lead to conflicts with existing libraries in the Anaconda environment, and import of the OVITO module may fail with an error related to the Qt framework. To fix this, please uninstall the ovito pip package by running 'pip uninstall -y ovito PySide6' and then install the OVITO Anaconda package provided by OVITO GmbH. Visit https://docs.ovito.org/python/introduction/installation.html for further instructions. If you would rather like to ignore this warning message, add the following code to the top of your Python script:

  import warnings
  warnings.filterwarnings('ignore', message='.*OVITO.*PyPI')

  import ovito._extensions.pyscript


# Config

In [2]:
# ITER_PATH = './Datasets/{train_or_test}_iter_{temperature}.pth'
PATH = './output/'
LOSS_PATH = PATH + 'loss/loss_{temperature}.txt'
RESULT_PATH = PATH + 'result/result_{temperature}.txt'
MODEL_PATH = PATH + 'model/{model}_{temperature}_{epoch}.pkl'
LOG_PATH = PATH + 'log/{date}_{information}.txt'
LOG_FIGURE_PATH = PATH + 'log/{date}_{information}.png'
DATA_PATH =  './data/'
TEMPERATURE = 400
LEARNING_RATE = 6e-3
# TRY_TIMES = 10
# TEMPERATURE = 150
# MAX = 1.0
# MIN = 0.5
# TIME_LENGTH = 50
STRIDE = 5
STEADY_LENGTH = 100
MIGRATION_LENGTH = 50
EMBEDDING_SIZE = 12
N_TRIAL = 100
INPUT_SIZE = 15
BATCH_SIZE = 16
EPOCHS = 1000

# DEVICE = torch.device('mps' if torch.cuda.is_available() else 'cpu')

DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

注意，还没对位置特征标准化

shape ： (num, time_length, feature)

In [3]:
def get_train_iter(
    Temperature, 
    Batch_size, 
    path
):
    # print(index)
    # print(data_num)
    
    pos_data, time_data = PosFeature(Temperature, f"{path}/temper.{Temperature}", 30)
    data = get_original_data(Temperature, time_data)
    # print(pos_data.shape)
    pos_data = pos_data.reshape(pos_data.shape[0], pos_data.shape[1], -1)
    data = np.concatenate(data, axis=2)
    # print(data.shape)
    # print(pos_data.shape)
    pas_data = np.concatenate([data, pos_data], axis=2)
    
    neg_pos_data, neg_time_data = NegFeature(Temperature, f"{path}/temper.{Temperature}", 30, len(time_data))
    neg_data = get_original_data(Temperature, neg_time_data)
    # print(neg_pos_data.shape)
    neg_pos_data = neg_pos_data.reshape(neg_pos_data.shape[0], neg_pos_data.shape[1], -1)
    
    neg_data = np.concatenate(neg_data, axis=2)
    neg_data = np.concatenate([neg_data, neg_pos_data], axis=2)
    
    pas_y  = np.ones((pas_data.shape[0], 1))
    neg_y  = np.zeros((neg_data.shape[0], 1))

    X = np.concatenate([pas_data, neg_data], axis=0)
    Y = np.concatenate([pas_y, neg_y], axis=0)


    X = torch.tensor(X, dtype=torch.float32)
    Y = torch.tensor(Y, dtype=torch.float32)
    X = X.unsqueeze(1)

    data_num = X.shape[0]
    shuffled_index = np.random.permutation(range(data_num))
    X = X[shuffled_index]
    Y = Y[shuffled_index]
    Train_X, Test_X = X[:int(data_num*0.8)], X[int(data_num*0.8):]
    Train_Y, Test_Y = Y[:int(data_num*0.8)], Y[int(data_num*0.8):]

    Train_generator = DataLoader(
        torch.utils.data.TensorDataset(Train_X, Train_Y), 
        Batch_size, 
        shuffle=True
    )
    Test_generator = DataLoader(
        torch.utils.data.TensorDataset(Test_X, Test_Y), 
        Batch_size, 
        shuffle=True
    )
    return Train_generator, Test_generator

# Model

In [4]:
class CNN(nn.Module):
    def __init__(
        self, 
        Channel_in, 
        Height_in, 
        Width_in, 
        Output_size, 
        Filter_num, 
        Kernel_list, 
        dropout = 0.5, 
    ):
        super(CNN, self).__init__()
        self.convs = nn.ModuleList([
            nn.Sequential(
                nn.Conv2d(
                    Channel_in, 
                    Filter_num, 
                    kernel_size=(kernel, Width_in), 
                    padding=((kernel - 1) // 2, 0), 
                ),
                nn.ReLU(),
                nn.MaxPool2d(
                    kernel_size=((Height_in+3)//4, 1), 
                    stride=(Height_in+3)//4, 
                    padding=((Height_in-Height_in//4*4+1)//2, 0), 
                ), 
            )
            for kernel in Kernel_list
        ])
        self.fc = nn.Sequential(
            nn.Dropout(dropout), 
            nn.Linear(Filter_num * len(Kernel_list) * 3, 64),
            nn.ReLU(), 
            nn.Linear(64, 16), 
            nn.Linear(16, Output_size)
        )

    def forward(self, x):
        out = [conv(x) for conv in self.convs]
        out = torch.cat(out, dim=1)
        out = out.view(x.size(0), -1)
        out = self.fc(out)
        return out


# Train and Val

In [5]:
def train(
    net:nn.Module, 
    Train_generator, 
    loss_func,
    optimizer, 
    scheduler, 
    device
):
    net = net.to(device)
    net.train()
    sum_loss = []
    
    Train_generator =  DataLoader(Train_generator.dataset, batch_size=BATCH_SIZE, shuffle=True, pin_memory=False)
    for x, y in Train_generator:
        x = x.to(device)
        y = y.to(device)
        optimizer.zero_grad()
        scores = net(x)
        loss = loss_func(scores, y)
        sum_loss.append(loss.item())
        loss.backward()
        optimizer.step()
        scheduler.step()
    
    return np.array(sum_loss).mean()
    
def evaluate(
    net:nn.Module, 
    Test_generator, 
    loss_func,
    optimizer, 
    scheduler, 
    device
):
    sum_loss = []
    net.eval()
    Test_generator = DataLoader(Test_generator.dataset, batch_size=BATCH_SIZE, shuffle=True, pin_memory=False)
    with torch.no_grad():
        for x, y in Test_generator:
            x = x.to(device)
            y = y.to(device)
            scores = net(x)
            loss = loss_func(scores, y)
            sum_loss.append(loss.item())
        
    return np.array(sum_loss).mean()

In [8]:
def train_multi_epochs(
    net:nn.Module, 
    Train_generator, 
    Test_generator,     
    loss_func,
    optimizer, 
    scheduler, 
    epochs, 
    device, 
    information:str, 
    show_train_process=None
):
    best_epoch = 0
    best_test_loss = 9.9e9
    best_net = net.state_dict()
    sum_train_loss, sum_test_loss = [], []
    date0 = time.strftime('%Y-%m-%d %a %H-%M-%S', time.localtime(time.time()))

    with open(LOG_PATH.format(date=date0, information=information), 'w') as log_fin:
        log_fin.write(information + '\n')
        log_fin.write('epoch' + ' ' + 'train_loss' + ' ' + 'test_loss' + ' ' + 'time' + ' ' + 'best_epoch' + '\n')
        t1 = time.time()
        for epoch in range(epochs):
            t0 = time.time()
            train_loss = train(net, Train_generator, loss_func, optimizer, scheduler, device).item()
            test_loss = evaluate(net, Test_generator, loss_func, optimizer, scheduler, device).item()

            sum_train_loss.append(train_loss)
            sum_test_loss.append(test_loss)

            if epoch == 0 or test_loss < best_test_loss:
                best_epoch = epoch
                best_test_loss = test_loss
                best_net = net.state_dict()

            log_fin.write(str(epoch) + ' ' + str(train_loss) + ' ' + str(test_loss) + ' ' + str(time.time()-t0) + ' ' + str(best_epoch) + '\n')
            if show_train_process != None and epoch % show_train_process == 0:
                print('epoch={:>4}, train_loss= {:.4f}, test_loss= {:.4f}, time= {:.2f}sec, best_epoch= {:>4}'.format(epoch, train_loss, test_loss, time.time()-t1, best_epoch))
                t1 = time.time()
        
        log_fin.write('\n')
        log_fin.write('best_epoch=' + str(best_epoch) + '\n')
        log_fin.write('best_test_loss=' + str(best_test_loss) + '\n')
    return best_test_loss, best_epoch, best_net, sum_train_loss, sum_test_loss




In [9]:
def singal_train(lr, ga, dropout=0.5):

    train_iter, test_iter = get_train_iter(TEMPERATURE, BATCH_SIZE, DATA_PATH)

    criteon = nn.BCEWithLogitsLoss().to(DEVICE)
    net = CNN(1, 5, 29, 1, 32, [9, 7, 5, 3], dropout)
    optimizer = optim.Adam(net.parameters(), lr=lr, weight_decay=1.0)
    scheduler = MultiStepLR(optimizer, [1000, 3000], gamma=ga, last_epoch=-1)
    
    best_test_loss, best_epoch, _, _, _ = train_multi_epochs(net, train_iter, test_iter, criteon, optimizer, scheduler, EPOCHS, DEVICE, 'lr = {:.8f}, gama = {:.4f}, dropout = {:4f}'.format(lr, ga, dropout), show_train_process=1)
    
    return best_test_loss

In [10]:
singal_train(LEARNING_RATE, 0.5, dropout=0.5)

epoch=   0, train_loss= 1.6952, test_loss= 0.7150, time= 0.09sec, best_epoch=    0
epoch=   1, train_loss= 0.7288, test_loss= 0.6678, time= 0.01sec, best_epoch=    1
epoch=   2, train_loss= 0.7011, test_loss= 0.6755, time= 0.01sec, best_epoch=    1
epoch=   3, train_loss= 0.6984, test_loss= 0.6792, time= 0.01sec, best_epoch=    1
epoch=   4, train_loss= 0.6969, test_loss= 0.6850, time= 0.01sec, best_epoch=    1
epoch=   5, train_loss= 0.6961, test_loss= 0.6864, time= 0.01sec, best_epoch=    1
epoch=   6, train_loss= 0.6946, test_loss= 0.6874, time= 0.01sec, best_epoch=    1
epoch=   7, train_loss= 0.6944, test_loss= 0.6886, time= 0.01sec, best_epoch=    1
epoch=   8, train_loss= 0.6945, test_loss= 0.6893, time= 0.01sec, best_epoch=    1
epoch=   9, train_loss= 0.6943, test_loss= 0.6912, time= 0.01sec, best_epoch=    1
epoch=  10, train_loss= 0.6936, test_loss= 0.6926, time= 0.01sec, best_epoch=    1
epoch=  11, train_loss= 0.6931, test_loss= 0.6937, time= 0.01sec, best_epoch=    1
epoc

0.6678171157836914